# Find Thai restaurants in San Francisco with 4.8+ stars and 30+ reviews

Learn how to use Exfunc to find best Thai restaurants in San Francisco.

## Getting Started

### Install packages

In [ ]:
!pip install exfunc

### Configure API keys

You will need to provide API keys. You can get your Exfunc API key [here](https://exfunc.com).

Ensure API key is accessible in your local environment.

In [ ]:
import getpass
import os

if "EXFUNC_API_KEY" not in os.environ:
    os.environ["EXFUNC_API_KEY"] = getpass.getpass("Exfunc API key:\n")

### Configure clients

In [ ]:
from exfunc import Exfunc

exfunc = Exfunc()

## Finding best rated Thai restaurants in San Francisco

### Search Thai restaurants in San Francisco

In [ ]:
num_pages = 3
business_ids = []
for i in range(num_pages):
    search_results = exfunc.yelp.search_businesses(request={
        "query": "thai",
        "location": "san francisco",
        "sort_by": "highest_rated",
        "start": 10*i,
    })
    business_ids += [business.business_id for business in search_results.businesses]

### Get business information

We will get business information by calling Yelp GetBusiness API for each business.

In [ ]:
from concurrent.futures import ThreadPoolExecutor

def get_business(business_id):
    try:
        response = exfunc.yelp.get_business(request={"business_id": business_id})
        return response.business
    except:
        return None

businesses = []
with ThreadPoolExecutor(max_workers=min(10, len(business_ids))) as executor:
    for business in executor.map(get_business, business_ids):
        if business:
            businesses.append(business)

### Filter businesses with 4.8+ rating and 30+ reviews

In [ ]:
filtered_businesses = []
for business in businesses:
    if business.rating and business.rating >= 4.8 and business.review_count and business.review_count >= 30:
        filtered_businesses.append(business)

### View results

In [ ]:
import pandas as pd

records = [business.model_dump() for business in filtered_businesses]

df = pd.DataFrame.from_records(records)
df